# Web Scraping Gold and Silver Prices
What's done?
* Using `lxml` to obtain the commodity dates and prices and convert to `pandas` dataframe. 
* 

In [119]:
from lxml import etree
import pandas as pd
import requests # for http requests
from datetime import datetime # change date format
import sys
def get_commodity_from_investing(commodity_name): # scrape `date` and `price` from investing.com
    url = "https://www.investing.com/commodities/%s-historical-data"%commodity_name
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}) # add User-Agent to prevent 403 error
    html = response.content
    tree = etree.HTML(html)

    rows = len(tree.xpath('/html/body/div[5]/section/div[9]/table[1]/tbody/tr[*]')) # use * to get all table rows 
    price = [0]*rows
    date = ['']*rows
    for i in range(rows):
        date[i] = date_formatting(tree.xpath('/html/body/div[5]/section/div[9]/table[1]/tbody/tr[%s]/td[1]/text()'%str(i+1))[0]) # index 0 to remove the list item
        price[i] = float(tree.xpath('/html/body/div[5]/section/div[9]/table[1]/tbody/tr[%s]/td[2]/text()'%str(i+1))[0].replace(',',''))
    #gold_date_price = list(zip(date,price)) # zip the two lists into one
    return date,price
def date_formatting(date): # convert Dec 07, 2018 to 2018-12-07
    objDate = datetime.strptime(str(date),'%b %d, %Y') # convert the input string to date objects from a known format
    date = datetime.strftime(objDate,'%Y-%m-%d') # convert the date object to any date format you like!!
    return date

gold_date,gold_price = get_commodity_from_investing('gold')
silver_date,silver_price = get_commodity_from_investing('silver')

# transform list into dataframe
df_gold = pd.DataFrame({'date':gold_date,'gold':gold_price}) 
df_silver = pd.DataFrame({'date':silver_date,'silver':silver_price})
df = df_silver.join(df_gold.set_index('date'),on='date') # join gold on silver since silver has more dates


Save and show the data results

In [120]:
df.to_csv('./data/data.csv')
print(df.head(7))


         date  silver    gold
0  2018-12-07  14.696  1252.6
1  2018-12-06  14.509  1243.6
2  2018-12-05  14.582  1242.6
3  2018-12-04  14.640  1246.6
4  2018-12-03  14.499  1239.6
5  2018-12-02  14.310     NaN
6  2018-11-30  14.217  1226.0


In [121]:
def getCommodityPrice(start_date,end_date,commodity):
    df = pd.DataFrame.from_csv('./data/data.csv')
    mask = (df['date']>=start_date) & (df['date']<=end_date) # alternatively, df[df.some_date.between(start_date, end_date)]
    df = df.loc[mask]
    return commodity, df[commodity].mean(), df[commodity].std()

In [132]:
import sys
start_date,end_date = '2018-12-04','2018-12-07'
getCommodityPrice(start_date,end_date,'gold')

# run commandline arg here
print(len(sys.argv))
print(sys.argv[0])
sys.argv[0] = "getCommodityPrice"
sys.argv[1] = "2017-05-01"
sys.argv[2] = "2017-05-03"
sys.argv[3] = 'gold'
sys.argv

3
getCommodityPrice


getCommodityPrice:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


IndexError: list assignment index out of range

# Reference
[Datetime formate](https://docs.python.org/3/library/datetime.html)